In [1]:
#Author: Nilesh Jakhotiya
#Email:  jakhotia.nilesh@gmail.com
from swat import CAS
import swat
from swat.cas.table import merge
import os
import dlpy
import numpy
import matplotlib.image as mping
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import pandas as pd

from dlpy.images import ImageTable
from dlpy.applications import UNet
from dlpy.model import Optimizer, Model, DataSpec, MomentumSolver, AdamSolver, StepLR
from dlpy.layers import *
from dlpy.utils import display_segmentation_images, create_segmentation_table, display_segmentation_results
from dlpy.sequential import Sequential

In [2]:
cashost='rdcgrd327'
casport=28041
casauth='C:/Users/nijakh/_authinfo'

# Start CAS session
conn = CAS(cashost, casport, authinfo=casauth, caslib="casuser")
conn.loadactionset('image')
conn.loadactionset('deeplearn')

NOTE: Added action set 'image'.
NOTE: Added action set 'deeplearn'.


[actionset]

 'deeplearn'

+ Elapsed: 0.1s, user: 0.108s, sys: 0.271s, mem: 11.5mb

In [3]:
# load_data
image_data=conn.loadTable( caslib="casuser",path="hmnist_28_28_RGB.csv", importoptions={"filetype":"csv"},casout={"name":"hmnist"})


NOTE: Cloud Analytic Services made the file hmnist_28_28_RGB.csv available as table HMNIST in caslib CASUSERHDFS(nijakh).


In [4]:
conn.table.tableInfo(name="HMNIST")     

,Name,Rows,Columns,IndexedColumns,Encoding,CreateTimeFormatted,ModTimeFormatted,AccessTimeFormatted,JavaCharSet,CreateTime,...,Repeated,View,MultiPart,SourceName,SourceCaslib,Compressed,Creator,Modifier,SourceModTimeFormatted,SourceModTime
0,HMNIST,10015,2353,0,utf-8,2020-05-03T10:06:33-04:00,2020-05-03T10:06:33-04:00,2020-05-03T10:06:33-04:00,UTF8,1.904134e+09,...,0,0,0,hmnist_28_28_RGB.csv,CASUSER(nijakh),0,nijakh,,2019-10-06T17:03:18-04:00,1.886015e+09


In [5]:
#Split data into train and test
conn.loadactionset('sampling')
conn.sampling.stratified(table="hmnist",target='label',samppct=30,samppct2=10,output={"casout":{"name":"hmnist_sample","replace":True},"copyvars":"ALL"})

NOTE: Added action set 'sampling'.
NOTE: The PARTIND option is set to be TRUE for segment stratified sampling.
NOTE: Stratified sampling is in effect.
NOTE: Using SEED=622165721 for sampling.


[OutputCasTables]

                 casLib           Name Label   Rows  Columns  \
 0  CASUSERHDFS(nijakh)  hmnist_sample        10015     2354   
 
                                             casTable  
 0  CASTable('hmnist_sample', caslib='CASUSERHDFS(...  

[PartIndMap]

 Partition Indicator Map
 
   PartIndName TargetName
 0  _PartInd1_      label

[STRAFreq]

 Frequencies
 
   TargetName TargetLevel   NObs  NSamp1  NSamp2
 0      label              10015    3004    1002

+ Elapsed: 5.15s, user: 14.2s, sys: 15.6s, mem: 4.9e+03mb

In [6]:
#split into Train, Validation and Test sets
conn.loadactionset('table')
conn.table.save(caslib="casuser",name="hmnist_train",replace=True,table={"name":'hmnist_sample',"where":"_partind1_=0" })
conn.table.save(caslib="casuser",name="hmnist_valid",replace=True,table={"name":'hmnist_sample',"where":"_partind1_=1" })
conn.table.save(caslib="casuser",name="hmnist_test",replace=True,table={"name":'hmnist_sample',"where":"_partind1_=2" })

NOTE: Added action set 'table'.
NOTE: Cloud Analytic Services saved the file hmnist_train.sashdat in caslib CASUSER(nijakh).
NOTE: Cloud Analytic Services saved the file hmnist_valid.sashdat in caslib CASUSER(nijakh).
NOTE: Cloud Analytic Services saved the file hmnist_test.sashdat in caslib CASUSER(nijakh).


[caslib]

 'CASUSER(nijakh)'

[name]

 'hmnist_test.sashdat'

+ Elapsed: 0.336s, user: 1.3s, sys: 0.689s, mem: 1.25e+03mb

In [7]:
# load_data
image_data=conn.loadTable( caslib="casuser",path="hmnist_train.sashdat", importoptions={"filetype":"hdat"},casout={"name":"train","replace":True})
image_data=conn.loadTable( caslib="casuser",path="hmnist_valid.sashdat", importoptions={"filetype":"hdat"},casout={"name":"valid","replace":True})
image_data=conn.loadTable( caslib="casuser",path="hmnist_test.sashdat", importoptions={"filetype":"hdat"},casout={"name":"test","replace":True})


NOTE: Cloud Analytic Services made the file hmnist_train.sashdat available as table TRAIN in caslib CASUSERHDFS(nijakh).
NOTE: Cloud Analytic Services made the file hmnist_valid.sashdat available as table VALID in caslib CASUSERHDFS(nijakh).
NOTE: Cloud Analytic Services made the file hmnist_test.sashdat available as table TEST in caslib CASUSERHDFS(nijakh).


In [8]:
conn.table.tableInfo(name="TEST")      
#conn.table.tableDetails(name="test")

,Name,Rows,Columns,IndexedColumns,Encoding,CreateTimeFormatted,ModTimeFormatted,AccessTimeFormatted,JavaCharSet,CreateTime,...,Repeated,View,MultiPart,SourceName,SourceCaslib,Compressed,Creator,Modifier,SourceModTimeFormatted,SourceModTime
0,TEST,1002,2354,0,utf-8,2020-05-03T10:06:47-04:00,2020-05-03T10:06:47-04:00,2020-05-03T10:06:47-04:00,UTF8,1.904134e+09,...,0,0,0,hmnist_test.sashdat,CASUSER(nijakh),0,nijakh,,2020-05-03T10:06:43-04:00,1.904134e+09


In [17]:
dlpy.applications.DenseNet121(conn, model_table='DENSENET121', n_classes=7, conv_channel=64, growth_rate=32, n_cells=[6, 12, 24, 16], n_channels=3, 
                              reduction=0.5, width=28, height=28, scale=1, random_flip='HV', random_crop='None', offsets=(103.939, 116.779, 123.68), random_mutation='random')


NOTE: Input layer added.
NOTE: Convolution layer added.
NOTE: Batch normalization layer added.
NOTE: Pooling layer added.
NOTE: Batch normalization layer added.
NOTE: Convolution layer added.
NOTE: Batch normalization layer added.
NOTE: Convolution layer added.
NOTE: Concatenation layer added.
NOTE: Batch normalization layer added.
NOTE: Convolution layer added.
NOTE: Batch normalization layer added.
NOTE: Convolution layer added.
NOTE: Concatenation layer added.
NOTE: Batch normalization layer added.
NOTE: Convolution layer added.
NOTE: Batch normalization layer added.
NOTE: Convolution layer added.
NOTE: Concatenation layer added.
NOTE: Batch normalization layer added.
NOTE: Convolution layer added.
NOTE: Batch normalization layer added.
NOTE: Convolution layer added.
NOTE: Concatenation layer added.
NOTE: Batch normalization layer added.
NOTE: Convolution layer added.
NOTE: Batch normalization layer added.
NOTE: Convolution layer added.
NOTE: Concatenation layer added.
NOTE: Batch n

NOTE: Model compiled successfully.


ExecutableNotFound: failed to execute ['dot', '-Tsvg'], make sure the Graphviz executables are on your systems' PATH

,Layer Id,Layer,Type,Kernel Size,Stride,Activation,Output Size,Number of Parameters,FLOPS(forward pass)
0,None,Input1,input,,,None,"(28, 28, 3)","(0, 0)",0
1,None,Convo.1,convo,"(7, 7)","(2, 2)",Identity,"(14, 14, 64)","(9408, 0)",1843968
2,None,B.N.1,batchnorm,,,Relu,"(14, 14, 64)","(0, 128)",0
3,None,Pool1,pool,"(3, 3)","(2, 2)",Max,"(7, 7, 64)","(0, 0)",0
4,None,B.N.2,batchnorm,,,Relu,"(7, 7, 64)","(0, 128)",0
...,...,...,...,...,...,...,...,...,...
301,None,Convo.120,convo,"(3, 3)","(1, 1)",Identity,"(1, 1, 32)","(36864, 0)",36864
302,None,Concat.58,concat,,,Identity,"(1, 1, 1024)","(0, 0)",0
303,None,B.N.121,batchnorm,,,Identity,"(1, 1, 1024)","(0, 2048)",0
304,None,Pool5,pool,"(1, 1)","(1, 1)",Mean,"(1, 1, 1024)","(0, 0)",0


In [18]:
conn.deepLearn.modelInfo(                        
modelTable={"name":"DENSENET121"}
)

,Descr,Value
0,Model Name,densenet121
1,Model Type,Convolutional Neural Network
2,Number of Layers,306
3,Number of Input Layers,1
4,Number of Output Layers,1
5,Number of Convolutional Layers,120
6,Number of Pooling Layers,5
7,Number of Fully Connected Layers,0
8,Number of Batch Normalization Layers,121
9,Number of Concatenation Layers,58


In [19]:
#create input variable list 
vars=[]
for x in range(2352):
    tmpx=str(x).zfill(4)
    vname="pixel{0}".format(tmpx)
    vars.append(vname)
    
#vars     

In [20]:
#Train the model
conn.deepLearn.dlTrain(
optimizer={"algorithm":{"method":"ADAM"}, "minibatchsize":50, "maxepochs":200},
inputs=vars,
modelTable={"name":"DENSENET121"},
modelWeights={"name":"hmnistTrainedWeights",
"replace":True},
forceequalpadding=True,
table={"name":"train"},
nominals=[{"name":"label"}],
target="label",
validtable={"name":"valid"})


,Descr,Value
0,Model Name,densenet121
1,Model Type,Convolutional Neural Network
2,Number of Layers,306
3,Number of Input Layers,1
4,Number of Output Layers,1
5,Number of Convolutional Layers,120
6,Number of Pooling Layers,5
7,Number of Fully Connected Layers,0
8,Number of Batch Normalization Layers,121
9,Number of Concatenation Layers,58


In [21]:
conn.deepLearn.dlTune(inputs=vars,
modelTable={"name":"DENSENET121"},
initweights={"name":"hmnistTrainedWeights"},
modelWeights={"name":"hmnistTrainedWeights2","replace":True},
forceequalpadding=True,
table={"name":"train"},
nominals=[{"name":"label"}],
target="label",
validtable={"name":"valid"},
optimizer={"algorithm":{"method":"ADAM","learningRate":{"lowerBound":1E-15, "upperBound":1E-1},"power":{"lowerBound":1E-15, "upperBound":1E-1},"gamma":{"lowerBound":.1, "upperBound":10}, "learningRatePolicy":"POLY"}, "dropout":{"lowerBound":.1, "upperBound":.5},"minibatchsize":{"lowerBound":30, "upperBound":90}, "maxepochs":200 } )

NOTE: Due to data distribution, miniBatchSize has been limited to 55.


,Descr,Value
0,Model Name,densenet121
1,Model Type,Convolutional Neural Network
2,Number of Layers,306
3,Number of Input Layers,1
4,Number of Output Layers,1
5,Number of Convolutional Layers,120
6,Number of Pooling Layers,5
7,Number of Fully Connected Layers,0
8,Number of Batch Normalization Layers,121
9,Number of Concatenation Layers,58


In [22]:
conn.deepLearn.dlScore(                
      casOut={"name":"test_scored",
              "replace":True},copyvars={"label"},
      initWeights={"name":"hmnistTrainedWeights2"},
      modelTable={"name":"DENSENET121"},
      table={"name":"test"}
)

[OutputCasTables]

                 casLib         Name  Rows  Columns  \
 0  CASUSERHDFS(nijakh)  test_scored  1002       11   
 
                                             casTable  
 0  CASTable('test_scored', caslib='CASUSERHDFS(ni...  

[ScoreInfo]

                          Descr         Value
 0  Number of Observations Read          1002
 1  Number of Observations Used          1002
 2  Misclassification Error (%)      26.54691
 3                   Loss Error      0.790936

+ Elapsed: 3.26s, user: 25.2s, sys: 70.1s, mem: 4.6e+04mb

In [15]:
#test_out=conn.CASTable('test_scored')
#test_out.fetch(maxrows=100)
conn.table.fetch(table="test_scored",maxrows=100,to=1000)

,label,_DL_P0_,_DL_P1_,_DL_P2_,_DL_P3_,_DL_P4_,_DL_P5_,_DL_P6_,_DL_PredName_,_DL_PredP_,_DL_PredLevel_
0,4.0,0.000424,0.975736,0.001226,0.000665,0.000083,0.000194,0.021672,4,0.975736,1.0
1,4.0,0.000394,0.985456,0.001111,0.000522,0.000054,0.000169,0.012294,4,0.985456,1.0
2,4.0,0.000918,0.993674,0.000197,0.000135,0.000007,0.000011,0.005058,4,0.993674,1.0
3,4.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4,1.000000,1.0
4,4.0,0.000584,0.990540,0.000748,0.000167,0.000004,0.000014,0.007944,4,0.990540,1.0
...,...,...,...,...,...,...,...,...,...,...,...
95,4.0,0.000606,0.982440,0.001035,0.000929,0.000016,0.000111,0.014864,4,0.982440,1.0
96,4.0,0.001036,0.972643,0.004509,0.000479,0.000020,0.000061,0.021252,4,0.972643,1.0
97,2.0,0.002952,0.484272,0.414501,0.001422,0.000734,0.000684,0.095436,4,0.484272,1.0
98,2.0,0.021549,0.340362,0.239397,0.060132,0.036044,0.016656,0.285859,4,0.340362,1.0


In [16]:
conn.loadactionset('freqtab')
conn.freqtab.freqtab(table="test_scored",tabulate=[{"vars":{"label"},"cross":{"_DL_PredName_"}}])

NOTE: Added action set 'freqtab'.


[LevelInfo]

 Variable Level Information
 
         Variable                                     Label  Levels  \
 0  _DL_PredName_  Predicted class with highest probability       5   
 1          label                                                 7   
 
                                               Values  
 0             0            1            2        ...  
 1                                      0 1 2 3 4 5 6  

[NObs]

 Number of Observations
 
       RowId                  Description   Value
 0     NREAD  Number of Observations Read  1002.0
 1     NUSED  Number of Observations Used  1002.0
 2  NWEIGHTS               Sum of Weights  1002.0

[Table1.CrossList]

 Table of label by _DL_PredName_
 
    F_label  label F__DL_PredName_ _DL_PredName_  Frequency     Percent  \
 0        0    0.0               0             0        2.0    0.199601   
 1        0    0.0               1             1       15.0    1.497006   
 2        0    0.0               2             2        2.0    0.199601   
 3        0    0.0               4             4       12.0    1.197605   
 4        0    0.0           Total                     31.0    3.093812   
 5        1    1.0               0             0        6.0    0.598802   
 6        1    1.0               1             1       26.0    2.594810   
 7        1    1.0               2             2       10.0    0.998004   
 8        1    1.0               4             4       13.0    1.297405   
 9        1    1.0           Total                     55.0    5.489022   
 10       2    2.0               0             0        2.0    0.199601   
 11       2    2.0               1             1       11.0    1.097804   
 12       2    2.0               2             2       39.0    3.892216   
 13       2    2.0               4             4       53.0    5.289421   
 14       2    2.0               6             6        7.0    0.698603   
 15       2    2.0           Total                    112.0   11.177645   
 16       3    3.0               0             0        1.0    0.099800   
 17       3    3.0               1             1        4.0    0.399202   
 18       3    3.0               2             2        1.0    0.099800   
 19       3    3.0               4             4        4.0    0.399202   
 20       3    3.0               6             6        1.0    0.099800   
 21       3    3.0           Total                     11.0    1.097804   
 22       4    4.0               1             1       10.0    0.998004   
 23       4    4.0               2             2       20.0    1.996008   
 24       4    4.0               4             4      615.0   61.377246   
 25       4    4.0               6             6       22.0    2.195609   
 26       4    4.0           Total                    667.0   66.566866   
 27       5    5.0               0             0        1.0    0.099800   
 28       5    5.0               1             1        6.0    0.598802   
 29       5    5.0               2             2        2.0    0.199601   
 30       5    5.0               4             4        7.0    0.698603   
 31       5    5.0               6             6        1.0    0.099800   
 32       5    5.0           Total                     17.0    1.696607   
 33       6    6.0               0             0        2.0    0.199601   
 34       6    6.0               1             1        2.0    0.199601   
 35       6    6.0               2             2       13.0    1.297405   
 36       6    6.0               4             4       66.0    6.586826   
 37       6    6.0               6             6       26.0    2.594810   
 38       6    6.0           Total                    109.0   10.878244   
 39   Total    NaN               0             0       14.0    1.397206   
 40   Total    NaN               1             1       74.0    7.385230   
 41   Total    NaN               2             2       87.0    8.682635   
 42   Total    NaN               4             4      770.